In [1]:
import pandas as pd
import cvxpy as cp
import numpy as np

In [10]:
# load the dataset
df = pd.read_csv("aoe4_data.csv")


In [11]:
# data cleaning and pre-processing
df.rename(columns={"Resource Cost": "Food", "Unnamed: 4": "Wood","Unnamed: 5": "Gold",}, inplace=True)
df.rename(columns={"Unnamed: 15" :"Spd","Unnamed: 16": "DPS","Unnamed: 17": "Rng"},inplace =True)
df.drop(columns=["Unnamed: 1", "Unnamed: 2","Bonus","Note","Unnamed: 19","Unnamed: 18","Unnamed: 13","Unnamed: 6","Move", "DPS", "HP per\nResource"], inplace=True)
df.rename(columns={" ": "Names"}, inplace=True)
df = df.iloc[:107]
df = df.dropna(subset=['Food', 'Gold', 'Wood'])
# we also drop the row where the unit is being listed twice, as it has different attack points to different units,
display(df.loc[df['Names']=="(Tusk)"].index)
display(df.loc[df['Names']=="(Tusk vs Building)"].index)
df = df.drop([50,60,51,61])
df = df.fillna(0)
df.head()

Index([50, 60], dtype='int64')

Index([51, 61], dtype='int64')

,Names,Food,Wood,Gold,Time,Pop,HP,Armor,Attack,Spd,Rng
2,Spearman,60.0,20.0,0.0,15.0,1.0,70.0,0,6,1.75,0.29
6,Landsknecht,60.0,0.0,100.0,22.0,1.0,80.0,0,17,1.25,0.29
9,Man-at-Arms,100.0,0.0,20.0,22.0,1.0,100.0,2,8,1.25,0.29
13,Palace Guard,100.0,0.0,20.0,22.0,1.0,155.0,3,12,1.25,0.29
16,Archer,30.0,50.0,0.0,15.0,1.0,70.0,0,5,1.5,5.00


In [12]:
# wood to gold ratio is 1:1.3
# food to gold ratio is also 1:1.3\n",
# we calculate the cost in gold.\n",
df.Food *= 1.3
df.Wood *= 1.3
df.Gold = df.Food + df.Wood + df.Gold
df.drop(columns=["Food", "Wood"], inplace=True)
df.head()

,Names,Gold,Time,Pop,HP,Armor,Attack,Spd,Rng
2,Spearman,104.0,15.0,1.0,70.0,0,6,1.75,0.29
6,Landsknecht,178.0,22.0,1.0,80.0,0,17,1.25,0.29
9,Man-at-Arms,150.0,22.0,1.0,100.0,2,8,1.25,0.29
13,Palace Guard,150.0,22.0,1.0,155.0,3,12,1.25,0.29
16,Archer,104.0,15.0,1.0,70.0,0,5,1.5,5.00


In [13]:
# Time, HealthPoint, Population Limit, Armor, Attack, Speed and Range would be our constraints\n",
# Time: total time to train these unit has to be less than certain units of time\n",
# HealthPoint: the total HealthPoint for the units have to be greater than certain threshold\n",
# similar to health points, total Armor, Attack points, speed and range has to be greater than certain threshold\n",
# population limit is the amount of units the country can host, so it has to be less than certain threshold\n",
c = np.array(df['Gold'])
A = np.array(df[['HP','Armor','Attack', "Spd", "Rng"]]).T
T = np.array(df["Time"]).T
P = np.array(df["Pop"]).T
df["Armor"] = df["Armor"].astype(float)
df["Attack"] = df["Attack"].astype(float)
df["Spd"] = df["Spd"].astype(float)
df["Rng"] = df["Rng"].astype(float)
b = np.array([5000,20,1000,100,130]) 
# These are our requirements

In [14]:
X = cp.Variable(df.shape[0])
obj = cp.Minimize(c @ X)
constraint_0 = [A@X>=b]
constraint_1 = [X>=0]
constraint_2 = [P@X <= 200]
constraint_3 = [T@X <= 1026]
prob = cp.Problem(obj,constraint_0+constraint_1+constraint_2+constraint_3)
prob.solve()
for i in range(df.shape[0]):
    if (abs(X.value.round(1)[i])):
        print(df.iloc[i]['Names'],abs(X.value.round(1)[i]))

Palace Guard 6.7
Longbowman 12.2
Khan 4.3
Junk 3.6
Lodya Demolition Ship 1.7


In [9]:
# additionaly, we are trying to restrict the possibility that certain unit dominates the composition of the army
constraint_4 = [X<=10]
prob = cp.Problem(obj,constraint_0+constraint_1+constraint_2+constraint_3+constraint_4)
prob.solve()
for i in range(df.shape[0]):
    if (abs(X.value.round(1)[i])):
        print(df.iloc[i]['Names'],abs(X.value.round(1)[i]))
# we further subdivide the units into specific groups:

Palace Guard 6.7
Archer 3.3
Longbowman 10.0
Khan 4.2
Junk 3.5
Lodya Demolition Ship 1.6


54